In [11]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegFileWriter
import seaborn as sns
sns.set_style('darkgrid')

# animation for paths of a Gaussian process
# kernel specifies the covariance function
# x-values from -xbnd to xbnd
# y-axis has values from -ybnd to ybnd
# saves gif if string is passed to name
def get_anim(kernel, xbnd=10., ybnd=2.5, name=None):
    nb_steps = 500
    delta_x = 2*xbnd / nb_steps
    x = np.arange(-xbnd, xbnd, delta_x)
    mean = np.zeros_like(x)
    cov = kernel(x.reshape(-1, 1))

    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure(num=' ', figsize=(10, 8))
    ax = plt.axes(xlim=(-xbnd, xbnd), ylim=(-ybnd, ybnd))
    ax.set_xlabel('x', fontsize=13)
    ax.set_ylabel('f(x)', fontsize=13)
    line, = ax.plot([], [], lw=2)

    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return line,

    # animation function.  This is called sequentially
    def animate(i):
        x = np.arange(-xbnd, xbnd, delta_x)
        samples = np.random.multivariate_normal(mean, cov)
        line.set_data(x, samples)
        return line,

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = FuncAnimation(fig, animate, init_func=init,
                         frames=50, interval=400, blit=True) #, repeat=False)
    if name is not None:
        anim.save(name + '.gif', writer=FFMpegFileWriter(fps=2)) #, dpi=200)
    return anim

# common kernels:
from sklearn.gaussian_process.kernels import DotProduct # linear kernel
from sklearn.metrics.pairwise import polynomial_kernel # polynomial kernel
from sklearn.gaussian_process.kernels import RBF # squared exponential kernel
from sklearn.gaussian_process.kernels import Matern # Matern kernel
from sklearn.gaussian_process.kernels import RationalQuadratic # rational quadratic kernel
from sklearn.gaussian_process.kernels import ExpSineSquared # periodic kernel
from sklearn.gaussian_process.kernels import ConstantKernel

# # example
# kernel = ConstantKernel() * RBF() 
# anim = get_anim(kernel, xbnd=4., ybnd=5.)

In [1]:
# Gaussian process posterior
def cond_distr(X1, y1, X2, kernel):
    """
    Calculate the posterior mean and covariance matrix for y2
    based on the corresponding input X2, the observations (y1, X1), 
    and the prior kernel function.
    """
    # Kernel of the observations
    Σ11 = kernel(X1, X1)
    # Kernel of observations vs to-predict
    Σ12 = kernel(X1, X2)
    # Solve
    solved = scipy.linalg.solve(Σ11, Σ12, assume_a='pos').T
    # Compute posterior mean
    μ2 = solved @ y1
    # Compute the posterior covariance
    Σ22 = kernel(X2, X2)
    Σ2 = Σ22 - (solved @ Σ12)
    return μ2[:, 0], Σ2  # mean, covariance

def _plot(mean, cov):
    fig= plt.figure(figsize=(14, 10))
    ax = plt.axes(xlim=(0, total_time), ylim=(-2.5, 2.5))
    ax.scatter(X, Y)
    plt.plot(t, np.sin(t), label='true function')
    plt.plot(t, mean, c='purple', label='mean prediction')
    plt.fill_between(t, mean - 1.96 * np.sqrt(np.diag(cov)), mean + 1.96 * np.sqrt(np.diag(cov)), 
                     color='purple', alpha=.25, label='credible interval')
    plt.legend()
    plt.show()

In [10]:
from IPython.display import display, clear_output
!pip install ipympl
clear_output()

%matplotlib inline
# %config InlineBackend.figure_formats = ['svg']
from ipywidgets import interact, interact_manual, FloatSlider
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegFileWriter
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel

# sample training data
X = np.array([1.01, 3.51, 4.51, 7.01, 7.91, 9.01]).reshape(-1, 1)
Y = np.sin(X)
nb_steps = 200
delta_t = 10 / nb_steps
t = np.arange(0, total_time, delta_t)

@interact(l=FloatSlider(value=1., min=1e-5, max=3, step=0.1, continuous_update=False), 
          sigma2=FloatSlider(value=1., min=1e-5, max=10, step=0.1, continuous_update=False))
def anim_hyper(l, sigma2):
    kernel = ConstantKernel(constant_value=sigma2) * RBF(length_scale=l) 
    mean, cov = cond_distr(X, Y, t.reshape(-1, 1), kernel) 
    _plot(mean, cov)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='l', max=3.0, min=1e-05), Fl…